# Normalization dr-masks

#Transformation Matrix from icv masks normalization are used to normalize dr-labels

In [ ]:
from nipype.interfaces import fsl
import os

In [ ]:
output_dir = './Praktikum/dr-masks_normalized'
ref_file = '/usr/share/data/fsl-mni152-templates/MNI152_T1_1mm_brain.nii.gz'
matrices_dir = './Praktikum/Pincram_all_icv_normalized'
dr_labels_dir = './Praktikum/dr_labels'

In [ ]:
sub_ids = [ f.split('.')[0] for f in os.listdir(dr_labels_dir) if f.endswith('.nii.gz')]
sub_ids

In [ ]:
for sub_id in sub_ids:
    print('--- {0} ---'.format(sub_id))
    flt = fsl.FLIRT(bins=640, cost_func='mutualinfo')
    flt.inputs.in_file = os.path.join(dr_labels_dir, sub_id + '.nii.gz')
    flt.inputs.reference = ref_file
    flt.inputs.in_matrix_file = os.path.join(matrices_dir, sub_id + '.mat')
    flt.inputs.out_file = os.path.join(output_dir, sub_id + '.nii.gz')
    flt.inputs.out_matrix_file = os.path.join(output_dir, sub_id + '.mat')
    print(flt.cmdline)
    res = flt.run()

# Sum patients with dementia and control patients

In [ ]:
import pandas as pd

In [ ]:
df = pd.read_excel('~/Downloads/diagnose.xlsx')
df.columns = ['id', 'diagnose', 'note']

In [ ]:
df['num'] = df['id'].apply(lambda x: x[8:12])

In [ ]:
df

In [ ]:
# Diagnose (0=HC, 1=MCI oder 2=AD) 	
control = df[df.diagnose == 0][['id', 'diagnose', 'num']]
mci = df[df.diagnose == 1][['id', 'diagnose', 'num']]
ad = df[df.diagnose == 2][['id', 'diagnose', 'num']]

# Help Functions

In [ ]:
def sum_voxels(img ,coord):
    # given a set of 1 voxel, sum it 
    data = img.get_data()
    for t in coord:
        x,y,z = t
        data[x,y,z] += 1
    #nib.save(img, dest)
    
def non_zero_voxels(img):
    # return a tuple of coordinates von voxel != 0
    x_dim, y_dim, z_dim = img.shape[0:3]
    data = img.get_data()
    l = []
    for x in range(x_dim):
        for y in range(y_dim):
            for z in range(z_dim):
                if data[x,y,z] != 0:
                    l.append((x,y,z))
    return tuple(l)

# SUM masks from control UNI images

In [ ]:
import nibabel as nib
import os
dest_file_name = 'control_UNI_sum.nii'
dest_path = './Praktikum/dr-masks_sum'
norm_masks_dir = './Praktikum/dr-masks_normalized'
dest = os.path.join(dest_path, dest_file_name)
suffix = '_UNI.nii.gz'
counter = 0
first = True
patients = len(control_nums)
for num in control_nums:
    img = os.path.join(norm_masks_dir, num + suffix)
    if first:
        print(img)
        mask = nib.load(img)
        first = False
    else:
        if os.path.isfile(img):
            print(nii)
            nii = nib.load(img)
            coord = non_zero_voxels(nii)
            sum_voxels(mask, coord)
nib.save(mask, dest)

# Mean

In [ ]:
sum_file_name = 'control_UNI_sum.nii'
dest_path = './Praktikum/dr-masks_sum'
sum_file = os.path.join(dest_path, sum_file_name)
mean_filename = 'control_UNI_mean.nii'
patients = len(control_nums)
dest = os.path.join(dest_path, mean_filename)
nii = nib.load(sum_file)
x_dim, y_dim, z_dim = nii.shape[0:3]
data = nii.get_data()
l = []
for x in range(x_dim):
    for y in range(y_dim):
        for z in range(z_dim):
            if data[x,y,z] != 0:
                d = data[x,y,z]/patients
                data[x,y,z] = 0 if d<=0.5 else 1
nib.save(nii, dest)

In [ ]:
# Sum mask for AD
ad = df[df.diagnose == 2][['id', 'diagnose', 'num']]
ad_nums = ad.num
ad_nums
dest_file_name = 'ad_UNI_sum.nii'
dest_path = './Praktikum/dr-masks_sum'
norm_masks_dir = './Praktikum/dr-masks_normalized'
dest = os.path.join(dest_path, dest_file_name)
suffix = '_UNI.nii.gz'
counter = 0
first = True
patients = len(ad_nums)
for num in ad_nums:
    img = os.path.join(norm_masks_dir, num + suffix)
    if first:
        print(img)
        mask = nib.load(img)
        first = False
    else:
        if os.path.isfile(img):
            print(nii)
            nii = nib.load(img)
            coord = non_zero_voxels(nii)
            sum_voxels(mask, coord)
nib.save(mask, dest)

In [ ]:
# Mean
sum_file_name = 'ad_UNI_sum.nii'
dest_path = './Praktikum/dr-masks_sum'
sum_file = os.path.join(dest_path, sum_file_name)
mean_filename = 'ad_UNI_mean.nii'
patients = len(ad_nums)
dest = os.path.join(dest_path, mean_filename)
nii = nib.load(sum_file)
x_dim, y_dim, z_dim = nii.shape[0:3]
data = nii.get_data()
l = []
for x in range(x_dim):
    for y in range(y_dim):
        for z in range(z_dim):
            if data[x,y,z] != 0:
                d = data[x,y,z]/patients
                data[x,y,z] = 0 if d<=0.5 else 1
nib.save(nii, dest)

In [ ]:
# Subtract
file_path = './Praktikum/dr-masks_sum'
ad_file_name = 'ad_UNI_mean.nii'
control_file_name = 'control_UNI_mean.nii'
dest_filename = 'ad_control_UNI_diff.nii'
ad_file = os.path.join(file_path, ad_file_name)
control_file = os.path.join(file_path, control_file_name)
dest = os.path.join(dest_path, dest_filename)
control = nib.load(control_file)
ad = nib.load(ad_file)
x_dim, y_dim, z_dim = control.shape[0:3] # they should be the same
control_data = control.get_data()
ad_data = ad.get_data()
l = []
for x in range(x_dim):
    for y in range(y_dim):
        for z in range(z_dim):
                control_data[x,y,z] = ad_data[x,y,z] - control_data[x,y,z]
nib.save(control, dest)

In [ ]:
control = df[df.diagnose == 0][['id', 'diagnose', 'num']]
# SUM masks from control UNI_DEN images
import nibabel as nib
import os
dest_file_name = 'control_UNI_DEN_sum.nii'
dest_path = './Praktikum/dr-masks_sum'
norm_masks_dir = './Praktikum/dr-masks_normalized'
dest = os.path.join(dest_path, dest_file_name)
suffix = '_UNI_DEN.nii.gz'
counter = 0
first = True
patients = len(control_nums)
for num in control_nums:
    img = os.path.join(norm_masks_dir, num + suffix)
    if first:
        print(img)
        mask = nib.load(img)
        first = False
    else:
        if os.path.isfile(img):
            print(nii)
            nii = nib.load(img)
            coord = non_zero_voxels(nii)
            sum_voxels(mask, coord)
nib.save(mask, dest)
# Mean
sum_file_name = 'control_UNI_DEN_sum.nii'
dest_path = './Praktikum/dr-masks_sum'
sum_file = os.path.join(dest_path, sum_file_name)
mean_filename = 'control_UNI_DEN_mean.nii'
patients = len(control_nums)
dest = os.path.join(dest_path, mean_filename)
nii = nib.load(sum_file)
x_dim, y_dim, z_dim = nii.shape[0:3]
data = nii.get_data()
l = []
for x in range(x_dim):
    for y in range(y_dim):
        for z in range(z_dim):
            if data[x,y,z] != 0:
                d = data[x,y,z]/patients
                data[x,y,z] = 0 if d<=0.5 else 1
nib.save(nii, dest)


In [ ]:
# Sum mask for AD
ad = df[df.diagnose == 2][['id', 'diagnose', 'num']]
ad_nums = ad.num
ad_nums
dest_file_name = 'ad_UNI_DEN_sum.nii'
dest_path = './Praktikum/dr-masks_sum'
norm_masks_dir = './Praktikum/dr-masks_normalized'
dest = os.path.join(dest_path, dest_file_name)
suffix = '_UNI_DEN.nii.gz'
counter = 0
first = True
patients = len(ad_nums)
for num in ad_nums:
    img = os.path.join(norm_masks_dir, num + suffix)
    if first:
        print(img)
        mask = nib.load(img)
        first = False
    else:
        if os.path.isfile(img):
            print(nii)
            nii = nib.load(img)
            coord = non_zero_voxels(nii)
            sum_voxels(mask, coord)
nib.save(mask, dest)
# Mean
sum_file_name = 'ad_UNI_DEN_sum.nii'
dest_path = './Praktikum/dr-masks_sum'
sum_file = os.path.join(dest_path, sum_file_name)
mean_filename = 'ad_UNI_DEN_mean.nii'
patients = len(ad_nums)
dest = os.path.join(dest_path, mean_filename)
nii = nib.load(sum_file)
x_dim, y_dim, z_dim = nii.shape[0:3]
data = nii.get_data()
l = []
for x in range(x_dim):
    for y in range(y_dim):
        for z in range(z_dim):
            if data[x,y,z] != 0:
                d = data[x,y,z]/patients
                data[x,y,z] = 0 if d<=0.5 else 1
nib.save(nii, dest)

In [ ]:
# Subtract
file_path = './Praktikum/dr-masks_sum'
ad_file_name = 'ad_UNI_DEN_mean.nii'
control_file_name = 'control_UNI_DEN_mean.nii'
dest_filename = 'ad_control_UNI_DEN_diff.nii'
ad_file = os.path.join(file_path, ad_file_name)
control_file = os.path.join(file_path, control_file_name)
dest = os.path.join(dest_path, dest_filename)
control = nib.load(control_file)
ad = nib.load(ad_file)
x_dim, y_dim, z_dim = control.shape[0:3] # they should be the same
control_data = control.get_data()
ad_data = ad.get_data()
l = []
for x in range(x_dim):
    for y in range(y_dim):
        for z in range(z_dim):
                control_data[x,y,z] = ad_data[x,y,z] - control_data[x,y,z]
nib.save(control, dest)

In [ ]:
# Same for  MCI patients

In [ ]:
mci = df[df.diagnose == 1][['id', 'diagnose', 'num']]
mci_nums = mci.num
mci_nums
dest_file_name = 'mci_UNI_sum.nii'
dest_path = './Praktikum/dr-masks_sum'
norm_masks_dir = './Praktikum/dr-masks_normalized'
dest = os.path.join(dest_path, dest_file_name)
suffix = '_UNI.nii.gz'
counter = 0
first = True
patients = len(mci_nums)
for num in mci_nums:
    img = os.path.join(norm_masks_dir, num + suffix)
    if first:
        print(img)
        mask = nib.load(img)
        first = False
    else:
        if os.path.isfile(img):
            print(nii)
            nii = nib.load(img)
            coord = non_zero_voxels(nii)
            sum_voxels(mask, coord)
nib.save(mask, dest)

# Mean
sum_file_name = 'mci_UNI_sum.nii'
dest_path = './Praktikum/dr-masks_sum'
sum_file = os.path.join(dest_path, sum_file_name)
mean_filename = 'mci_UNI_mean.nii'
patients = len(mci_nums)
dest = os.path.join(dest_path, mean_filename)
nii = nib.load(sum_file)
x_dim, y_dim, z_dim = nii.shape[0:3]
data = nii.get_data()
l = []
for x in range(x_dim):
    for y in range(y_dim):
        for z in range(z_dim):
            if data[x,y,z] != 0:
                d = data[x,y,z]/patients
                data[x,y,z] = 0 if d<=0.5 else 1
nib.save(nii, dest)

# Subtract
file_path = './Praktikum/dr-masks_sum'
mci_file_name = 'mci_UNI_mean.nii'
control_file_name = 'control_UNI_mean.nii'
dest_filename = 'mci_control_UNI_diff.nii'
mci_file = os.path.join(file_path, mci_file_name)
control_file = os.path.join(file_path, control_file_name)
dest = os.path.join(dest_path, dest_filename)
control = nib.load(control_file)
mci = nib.load(mci_file)
x_dim, y_dim, z_dim = control.shape[0:3] # they should be the same
control_data = control.get_data()
mci_data = mci.get_data()
l = []
for x in range(x_dim):
    for y in range(y_dim):
        for z in range(z_dim):
                control_data[x,y,z] = mci_data[x,y,z] - control_data[x,y,z]
nib.save(control, dest)

In [ ]:
mci = df[df.diagnose == 1][['id', 'diagnose', 'num']]
mci_nums = mci.num
mci_nums
dest_file_name = 'mci_UNI_DEN_sum.nii'
dest_path = './Praktikum/dr-masks_sum'
norm_masks_dir = './Praktikum/dr-masks_normalized'
dest = os.path.join(dest_path, dest_file_name)
suffix = '_UNI_DEN.nii.gz'
counter = 0
first = True
patients = len(mci_nums)
for num in mci_nums:
    img = os.path.join(norm_masks_dir, num + suffix)
    if first:
        print(img)
        mask = nib.load(img)
        first = False
    else:
        if os.path.isfile(img):
            print(nii)
            nii = nib.load(img)
            coord = non_zero_voxels(nii)
            sum_voxels(mask, coord)
nib.save(mask, dest)

# Mean
sum_file_name = 'mci_UNI_DEN_sum.nii'
dest_path = './Praktikum/dr-masks_sum'
sum_file = os.path.join(dest_path, sum_file_name)
mean_filename = 'mci_UNI_DEN_mean.nii'
patients = len(mci_nums)
dest = os.path.join(dest_path, mean_filename)
nii = nib.load(sum_file)
x_dim, y_dim, z_dim = nii.shape[0:3]
data = nii.get_data()
l = []
for x in range(x_dim):
    for y in range(y_dim):
        for z in range(z_dim):
            if data[x,y,z] != 0:
                d = data[x,y,z]/patients
                data[x,y,z] = 0 if d<=0.5 else 1
nib.save(nii, dest)

# Subtract
file_path = './Praktikum/dr-masks_sum'
mci_file_name = 'mci_UNI_DEN_mean.nii'
control_file_name = 'control_UNI_DEN_mean.nii'
dest_filename = 'mci_control_UNI_DEN_diff.nii'
mci_file = os.path.join(file_path, mci_file_name)
control_file = os.path.join(file_path, control_file_name)
dest = os.path.join(dest_path, dest_filename)
control = nib.load(control_file)
mci = nib.load(mci_file)
x_dim, y_dim, z_dim = control.shape[0:3] # they should be the same
control_data = control.get_data()
mci_data = mci.get_data()
l = []
for x in range(x_dim):
    for y in range(y_dim):
        for z in range(z_dim):
                control_data[x,y,z] = mci_data[x,y,z] - control_data[x,y,z]
nib.save(control, dest)